In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
from dash import dash_table
import json
import glob



In [2]:
# Load workers data and format correctly
workers_df = pd.read_json('extracted/workers.json').transpose().reset_index()
workers_df.columns = ['worker_id', 'name', 'base_salary']



In [3]:
# Load prices.json as a dictionary and convert to DataFrame
with open('prices.json') as f:
    prices_data = json.load(f)
prices_df = pd.DataFrame(list(prices_data.items()), columns=['technical_problem', 'price'])



In [4]:
# Function to load and flatten data from multiple JSON files in a directory
def load_and_flatten_data(directory, key_field):
    records = []
    for file_path in glob.glob(f'{directory}/*.json'):
        with open(file_path) as f:
            data = json.load(f)
        for location, calls in data.items():
            for call_id, call_info in calls.items():
                call_info['call_id'] = call_id
                call_info['location'] = location
                records.append(call_info)
    return pd.DataFrame(records)



In [5]:
# Load feature calls, previous calls, previous reports, and schedules
feature_calls_df = load_and_flatten_data('extracted/feature_calls', 'call_id')
previous_calls_df = load_and_flatten_data('extracted/previous_calls', 'call_id')


,date,tlf_number,technical_problem,difficulty,commission,call_id,location,price_x,price_y
0,2024-10-23 16:54:39.515508,72345741,teams_problems,medium,665.0,c_5e239e86-20f4-4053-a77b-a01dd3971aad,bangalore,665,665
1,2024-10-23 12:18:18.515647,53336166,basic_hardware_troubleshooting,easy,576.0,c_1cbe224b-804b-4b30-bf78-198feea748f8,bangalore,720,720
2,2024-10-20 01:42:41.515693,81959025,email_related_issues,medium,700.0,c_ca2f35cc-47fb-446f-a8ad-3c0360edb98c,bangalore,700,700
3,2024-10-24 12:14:40.515734,96315388,device_and_peripheral_setup,medium,705.0,c_59b1c1cc-6362-4edd-aed8-720536d733e6,bangalore,705,705
4,2024-10-24 19:24:07.515775,54054590,internet_problems,medium,680.0,c_69e87302-cad8-42a3-9fb2-e64c0a4b14e4,bangalore,680,680
...,...,...,...,...,...,...,...,...,...
257096,2024-10-22 17:11:21.910960,35675011,zoom_problems,hard,834.0,c_2b052e28-f48f-4c85-8b75-f73f27ef75f0,hyderabad,695,695
257097,2024-10-23 09:14:03.910999,22767682,internet_problems,medium,680.0,c_65dc01b2-5091-40d1-9838-c8a749b3f766,hyderabad,680,680
257098,2024-10-19 17:31:44.911038,24957531,email_related_issues,medium,700.0,c_827e75d4-1948-433a-ad5d-f2bdf083ec71,hyderabad,700,700
257099,2024-10-20 17:10:37.911077,00563223,internet_problems,medium,680.0,c_68b59f6b-00e7-4529-a353-d9ded93226ce,hyderabad,680,680


In [27]:
previous_calls_df

,date,tlf_number,technical_problem,difficulty,commission,call_id,location,price_x,price_y
0,2024-10-19 07:28:55.254686,85062036,zoom_problems,medium,695.0,c_a317abc5-a7e1-40c3-b486-c4eefdf8be05,bangalore,695,695
1,2024-10-22 22:08:18.254841,73320532,browser_and_web_based_support,medium,650.0,c_78d3cde6-0da9-49f1-91b8-907b180b30a2,bangalore,650,650
2,2024-10-26 08:29:09.254885,97493770,browser_and_web_based_support,easy,520.0,c_130313ec-a141-4306-825b-826c63096cff,bangalore,650,650
3,2024-10-25 07:23:24.254929,64380401,teams_problems,easy,532.0,c_4027388e-7996-4fc2-aef6-c99d66e22b09,bangalore,665,665
4,2024-10-22 19:47:52.254979,43520611,account_and_security_issues,hard,852.0,c_2d36d2d7-d07f-4431-a699-e0ac84a0b7ce,bangalore,710,710
...,...,...,...,...,...,...,...,...,...
283110,2024-10-22 13:46:29.911955,65079230,browser_and_web_based_support,easy,520.0,c_3f62128c-810d-4817-ab92-52ecbd3c4c2a,hyderabad,650,650
283111,2024-10-24 03:32:40.911993,17109055,email_related_issues,easy,560.0,c_b227239a-5a76-4e2b-842a-c7d7f71e9998,hyderabad,700,700
283112,2024-10-19 19:37:55.912031,04702329,account_and_security_issues,easy,568.0,c_afbae2d8-b5a8-49f1-8037-8f4e628ac620,hyderabad,710,710
283113,2024-10-25 08:18:19.912069,34131698,cloud_and_storage_solutions,hard,828.0,c_d0dac5b4-3792-46e3-908b-7a67730d8bc3,hyderabad,690,690


In [6]:

# Load previous and new reports
previous_reports = []
for file_path in glob.glob('extracted/previous_reports/*.json'):
    previous_reports.extend(pd.read_json(file_path).to_dict(orient='records'))
previous_reports_df = pd.DataFrame(previous_reports)



In [15]:
new_reports = []
for file_path in glob.glob('ikkeheltimal_call_reports_11_20/future_call_reports/*.json'):
    new_reports.extend(pd.read_json(file_path).to_dict(orient='records'))
new_reports_df = pd.DataFrame(new_reports)



In [16]:
# Combine prices with calls based on technical problems
feature_calls_df = feature_calls_df.merge(prices_df, on='technical_problem', how='left')
previous_calls_df = previous_calls_df.merge(prices_df, on='technical_problem', how='left')



In [11]:
previous_reports_df

,call_id,worker_id,call_time,likely_to_recommend,professional_score,call_profit
0,c_a317abc5-a7e1-40c3-b486-c4eefdf8be05,w_3cbcf16f-2173-4281-bad0-b5579ba0f123,57.057332,1.0,NaN,695
1,c_256844cb-b710-4da7-8f95-238a7fdd5261,w_3cbcf16f-2173-4281-bad0-b5579ba0f123,69.294334,1.0,NaN,735
2,c_46bfdc38-5130-4e08-9918-4851d84930ce,w_3cbcf16f-2173-4281-bad0-b5579ba0f123,70.794334,1.0,NaN,735
3,c_497ac5ca-8ff3-4773-968c-2e300486c35d,w_3cbcf16f-2173-4281-bad0-b5579ba0f123,54.379299,1.0,NaN,532
4,c_97384374-b420-4259-aaa9-f2e1571928cb,w_3cbcf16f-2173-4281-bad0-b5579ba0f123,77.457332,1.0,NaN,834
...,...,...,...,...,...,...
72511,c_282338ea-b4a3-4d55-801c-1935c08be259,w_ff6501e8-b49a-4512-a9eb-e06565d21355,37.453295,1.0,NaN,705
72512,c_9f076c7e-aa1f-44e2-a904-ec7daaec52c4,w_ff6501e8-b49a-4512-a9eb-e06565d21355,62.093346,1.0,NaN,780
72513,c_bece22d1-0ba0-4208-91f8-366c8267177c,w_ff6501e8-b49a-4512-a9eb-e06565d21355,61.482360,1.0,NaN,864
72514,c_ac55b479-d0ad-4c68-a637-87b3c360ceb9,w_ff6501e8-b49a-4512-a9eb-e06565d21355,32.353295,1.0,NaN,564


In [17]:
new_reports_df.head()

,call_id,worker_id,call_time,likely_to_recommend,professional_score,call_profit
0,c_670f54f4-9041-4ae2-85b9-05d30a4dbf23,w_eb5ca7e7-197b-4128-9cdd-17b8d7d07803,33.017543,1.0,NaN,780
1,c_a4b4e43f-5939-4992-b576-c844f90a8131,w_eb5ca7e7-197b-4128-9cdd-17b8d7d07803,26.193912,1.0,NaN,900
2,c_e2a563ff-802e-4f56-9d08-47fadc3270b6,w_eb5ca7e7-197b-4128-9cdd-17b8d7d07803,26.551302,1.0,NaN,864
3,c_c2f70a56-60d8-45b8-b90c-73a6c24d6cc8,w_eb5ca7e7-197b-4128-9cdd-17b8d7d07803,28.051302,1.0,NaN,864
4,c_17d86ada-c033-460e-9205-47b5c1e1f248,w_eb5ca7e7-197b-4128-9cdd-17b8d7d07803,31.855709,1.0,NaN,846


In [18]:
# Calculate key performance metrics
def calculate_metrics(report_df):
    return {
        "Average Profit": report_df['call_profit'].mean(),
        "Average Call Time": report_df['call_time'].mean(),
        "Average Recommendation": report_df['likely_to_recommend'].mean(),
        
    }



In [44]:
new_reports_df

,call_id,worker_id,call_time,likely_to_recommend,professional_score,call_profit
0,c_670f54f4-9041-4ae2-85b9-05d30a4dbf23,w_eb5ca7e7-197b-4128-9cdd-17b8d7d07803,33.017543,1.0,NaN,780
1,c_a4b4e43f-5939-4992-b576-c844f90a8131,w_eb5ca7e7-197b-4128-9cdd-17b8d7d07803,26.193912,1.0,NaN,900
2,c_e2a563ff-802e-4f56-9d08-47fadc3270b6,w_eb5ca7e7-197b-4128-9cdd-17b8d7d07803,26.551302,1.0,NaN,864
3,c_c2f70a56-60d8-45b8-b90c-73a6c24d6cc8,w_eb5ca7e7-197b-4128-9cdd-17b8d7d07803,28.051302,1.0,NaN,864
4,c_17d86ada-c033-460e-9205-47b5c1e1f248,w_eb5ca7e7-197b-4128-9cdd-17b8d7d07803,31.855709,1.0,NaN,846
...,...,...,...,...,...,...
22062,c_8905e65e-ef82-419c-a0c3-96e74589791e,w_dd684140-be47-455f-a00f-0c21fb54752e,33.134185,1.0,NaN,852
22063,c_b80b05f3-cf5b-47f3-a25f-0219e06dea0c,w_dd684140-be47-455f-a00f-0c21fb54752e,44.738654,1.0,NaN,864
22064,c_593c7020-4e2f-4e01-875f-a156134d02db,w_dd684140-be47-455f-a00f-0c21fb54752e,47.352038,1.0,NaN,834
22065,c_2aa0b6d7-9b7c-4d80-bc35-547021944abf,w_dd684140-be47-455f-a00f-0c21fb54752e,46.770240,1.0,NaN,816


In [43]:
feature_calls_df

,date,tlf_number,technical_problem,difficulty,commission,call_id,location,price_x,price_y,worker_id_x,...,worker_id_feature,call_time_feature,likely_to_recommend_feature,professional_score_feature,call_profit_feature,worker_id_new,call_time_new,likely_to_recommend_new,professional_score_new,call_profit_new
0,2024-10-22 06:53:22.970591,77268423,browser_and_web_based_support,hard,780.0,c_670f54f4-9041-4ae2-85b9-05d30a4dbf23,mumbai,650,650,w_eb5ca7e7-197b-4128-9cdd-17b8d7d07803,...,w_eb5ca7e7-197b-4128-9cdd-17b8d7d07803,33.017543,1.0,NaN,780,w_eb5ca7e7-197b-4128-9cdd-17b8d7d07803,33.017543,1.0,NaN,780
1,2024-10-21 14:45:05.984932,97703985,software_installation_and_configuration,hard,900.0,c_a4b4e43f-5939-4992-b576-c844f90a8131,mumbai,750,750,w_eb5ca7e7-197b-4128-9cdd-17b8d7d07803,...,w_eb5ca7e7-197b-4128-9cdd-17b8d7d07803,26.193912,1.0,NaN,900,w_eb5ca7e7-197b-4128-9cdd-17b8d7d07803,26.193912,1.0,NaN,900
2,2024-10-24 22:13:30.140198,48349003,basic_hardware_troubleshooting,hard,864.0,c_e2a563ff-802e-4f56-9d08-47fadc3270b6,mumbai,720,720,w_eb5ca7e7-197b-4128-9cdd-17b8d7d07803,...,w_eb5ca7e7-197b-4128-9cdd-17b8d7d07803,26.551302,1.0,NaN,864,w_eb5ca7e7-197b-4128-9cdd-17b8d7d07803,26.551302,1.0,NaN,864
3,2024-10-24 07:17:22.161072,16410648,basic_hardware_troubleshooting,hard,864.0,c_c2f70a56-60d8-45b8-b90c-73a6c24d6cc8,mumbai,720,720,w_eb5ca7e7-197b-4128-9cdd-17b8d7d07803,...,w_eb5ca7e7-197b-4128-9cdd-17b8d7d07803,28.051302,1.0,NaN,864,w_eb5ca7e7-197b-4128-9cdd-17b8d7d07803,28.051302,1.0,NaN,864
4,2024-10-19 22:50:24.224873,85901720,device_and_peripheral_setup,hard,846.0,c_17d86ada-c033-460e-9205-47b5c1e1f248,mumbai,705,705,w_eb5ca7e7-197b-4128-9cdd-17b8d7d07803,...,w_eb5ca7e7-197b-4128-9cdd-17b8d7d07803,31.855709,1.0,NaN,846,w_eb5ca7e7-197b-4128-9cdd-17b8d7d07803,31.855709,1.0,NaN,846
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22062,2024-10-23 05:30:14.522715,81994133,account_and_security_issues,hard,852.0,c_8905e65e-ef82-419c-a0c3-96e74589791e,hyderabad,710,710,w_dd684140-be47-455f-a00f-0c21fb54752e,...,w_dd684140-be47-455f-a00f-0c21fb54752e,33.134185,1.0,NaN,852,w_dd684140-be47-455f-a00f-0c21fb54752e,33.134185,1.0,NaN,852
22063,2024-10-20 02:49:27.574552,90285020,basic_hardware_troubleshooting,hard,864.0,c_b80b05f3-cf5b-47f3-a25f-0219e06dea0c,hyderabad,720,720,w_dd684140-be47-455f-a00f-0c21fb54752e,...,w_dd684140-be47-455f-a00f-0c21fb54752e,44.738654,1.0,NaN,864,w_dd684140-be47-455f-a00f-0c21fb54752e,44.738654,1.0,NaN,864
22064,2024-10-19 10:44:36.693101,18132262,zoom_problems,hard,834.0,c_593c7020-4e2f-4e01-875f-a156134d02db,hyderabad,695,695,w_dd684140-be47-455f-a00f-0c21fb54752e,...,w_dd684140-be47-455f-a00f-0c21fb54752e,47.352038,1.0,NaN,834,w_dd684140-be47-455f-a00f-0c21fb54752e,47.352038,1.0,NaN,834
22065,2024-10-19 10:39:55.757160,94440452,internet_problems,hard,816.0,c_2aa0b6d7-9b7c-4d80-bc35-547021944abf,hyderabad,680,680,w_dd684140-be47-455f-a00f-0c21fb54752e,...,w_dd684140-be47-455f-a00f-0c21fb54752e,46.770240,1.0,NaN,816,w_dd684140-be47-455f-a00f-0c21fb54752e,46.770240,1.0,NaN,816


In [24]:
# Calculate metrics for previous and new reports
previous_metrics = calculate_metrics(previous_reports_df)
new_metrics = calculate_metrics(new_reports_df)



In [25]:
previous_metrics

{'Average Profit': np.float64(698.6554415577252),
 'Average Call Time': np.float64(44.53929245603706),
 'Average Recommendation': np.float64(1.5461787743394562)}

In [20]:
# Calculate percentage changes
comparison = {
    "Metric": ["Average Profit", "Average Call Time", "Average Recommendation"],
    "Previous": [previous_metrics["Average Profit"], previous_metrics["Average Call Time"], previous_metrics["Average Recommendation"]],
    "New": [new_metrics["Average Profit"], new_metrics["Average Call Time"], new_metrics["Average Recommendation"]],
    "Change (%)": [
        (new_metrics["Average Profit"] - previous_metrics["Average Profit"]) / previous_metrics["Average Profit"] * 100 if previous_metrics["Average Profit"] else None,
        (new_metrics["Average Call Time"] - previous_metrics["Average Call Time"]) / previous_metrics["Average Call Time"] * 100 if previous_metrics["Average Call Time"] else None,
        (new_metrics["Average Recommendation"] - previous_metrics["Average Recommendation"]) / previous_metrics["Average Recommendation"] * 100 if previous_metrics["Average Recommendation"] else None
    ]
}

# Create a DataFrame for comparison
comparison_df = pd.DataFrame(comparison)

In [26]:
comparison_df

,Metric,Previous,New,Change (%)
0,Average Profit,698.655442,814.616305,16.597718
1,Average Call Time,44.539292,51.851774,16.418046
2,Average Recommendation,1.546179,1.459705,-5.592771
